In [101]:
import pandas as pd
import numpy as np
import seaborn as sns
from io import StringIO

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

import re
import os
import tqdm
import string

In [2]:
import re
import os
from io import StringIO
def clean_text(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', str(text))
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace


def remove_stops(text, stops):
    text = re.sub(r"AC\/\d{1,4}\/\d{1,4}", "", str(text))
    words = text.split()
    final = []
    for word in words:
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    final = "".join([i for i in final if not i.isdigit()])
    while "  " in final:
        final = final.replace("  ", " ")
    return (final)

In [3]:
os.chdir('/Users/alisdghnia/Desktop/')

In [4]:
df1 = pd.read_excel('Full Dataset Cleaned with Excel Calculations.xlsx')

In [5]:
df1['Name'] = df1['Name'].apply(lambda x: x.replace('.pdf', ''))

In [6]:
df3 = pd.read_csv('Files Texts Dataframe.csv')

In [7]:
df3 = df3.rename(columns= {'Unnamed: 0':'File Name'})

In [8]:
new_corpus = []
for i in df3['Text Corpus']:
    i = clean_text(i)
    i = remove_stops(i, 'english')
    new_corpus.append(i)

df3['New Corpus'] = new_corpus

In [9]:
df3 = df3.drop(columns=['Text Corpus'])

In [10]:
df = pd.merge(df1, df3, left_on = 'Name', right_on= 'File Name')

In [11]:
df = df.drop(columns= ['Unnamed: 0', 'Unnamed: 19', 'Unnamed: 21','Unnamed: 23', 'Circulating Supply.1', 'Market Cap.1', 'Volume in Currency (24Hr).1', 'Circulating Supply',
            'Summary Section', 'Reference Section', 'Abstract Section'])

In [12]:
tfidf = TfidfVectorizer(max_df=0.01, min_df=5, lowercase= True, stop_words='english')
features = tfidf.fit_transform(df['New Corpus']).toarray()
words = tfidf.get_feature_names()
# labels = df.category_id
print(features.shape)

tfidf_df = pd.DataFrame(features, columns=words)

(556, 1565)


/Users/alisdghnia/.conda/envs/pythonProject/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
print(np.median(df['Price (Intraday)']))
print(np.median(df['Market Cap']))
print(np.median(df['Volume in Currency (24Hr)']))

0.023677499999999997
2149000.0
80065.0


In [14]:
successful_price = []
value_price = []
# successcol = df[['Price (Intraday)', 'Market Cap', 'Volume in Currency (24Hr)']]
for i in df['Price (Intraday)']:
    if i > np.median(df['Price (Intraday)']):
        value_price = 1
    else:
        value_price = 0
    # print(value)
    successful_price.append(value_price)

# successful_price

In [15]:
successful_market = []
value_market = []
# successcol = df[['Price (Intraday)', 'Market Cap', 'Volume in Currency (24Hr)']]
for i in df['Market Cap']:
    if i > np.median(df['Market Cap']):
        value_market = 1
    else:
        value_market = 0
    # print(value)
    successful_market.append(value_market)

# successful_market

In [16]:
successful_volume = []
value_volume = []
# successcol = df[['Price (Intraday)', 'Market Cap', 'Volume in Currency (24Hr)']]
for i in df['Volume in Currency (24Hr)']:
    if i > np.median(df['Volume in Currency (24Hr)']):
        value_volume = 1
    else:
        value_volume = 0
    # print(value)
    successful_volume.append(value_volume)

# successful_volume

In [17]:
df['Successful (Price)'] = successful_price
df['Successful (Market Cap)'] = successful_market
df['Successful (Volume)'] = successful_volume

In [18]:
new_col = ['Successful (Price)', 'Successful (Market Cap)', 'Successful (Volume)','Name', 'File Size (MB)', 'Number of Pages', 'Word Count', 'Flesch Reading Ease', 'New Dale-Chall', 'Crypto Name', 
            'Abstract Int', 'Summary Int', 'Reference Int', 'Price (Intraday)','Market Cap', 'Volume in Currency (24Hr)', 'File Name', 'New Corpus']
            
df = df[new_col]
df = df.reindex(columns= new_col)
df

,Successful (Price),Successful (Market Cap),Successful (Volume),Name,File Size (MB),Number of Pages,Word Count,Flesch Reading Ease,New Dale-Chall,Crypto Name,Abstract Int,Summary Int,Reference Int,Price (Intraday),Market Cap,Volume in Currency (24Hr),File Name,New Corpus
0,0,0,0,WeTrust1,1.342260,40,12504,43.02,7.33,WeTrust USD,1,0,1,2.483000e-03,228813.0,3.0,WeTrust1,wetrust whitepaper table of contents abstract ...
1,1,0,0,Libra4,0.293671,41,17474,54.42,6.51,Libera USD,1,0,1,4.740480e-01,0.0,131.0,Libra4,state machine replication in the libra blockch...
2,0,0,0,B2BX1,15.404950,42,1,121.22,19.48,X USD,0,0,0,6.870000e-04,0.0,9220.0,B2BX1,
3,0,1,1,Bytom1,0.374002,17,4536,34.15,7.79,Bytom USD,1,0,1,1.237500e-02,21831000.0,1088000.0,Bytom1,bytomv bytom an interoperation protocol for di...
4,0,0,0,Cindicator1,1.123809,37,15509,42.92,7.08,Cindicator USD,0,0,0,6.710000e-04,1342000.0,65.0,Cindicator1,hybrid intelligence for eective asset managem...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,0,0,1,CPChain1,1.195266,34,7203,40.28,6.96,CPChain USD,0,0,0,3.619000e-03,1363000.0,111473.0,CPChain1,cyberphysical chain cpchain whitepaper decentr...
552,1,1,1,Presearch2,5.291211,39,12904,41.70,7.16,Presearch USD,1,0,0,8.353100e-02,33127000.0,107073.0,Presearch2,whitepaper the communitypowered search engine ...
553,0,0,0,Electra1,0.624846,34,5508,37.13,9.05,Electra USD,1,1,0,4.600000e-07,0.0,48.0,Electra1,white paper release january table of contents ...
554,0,0,0,Blockmason1,3.355893,31,13640,49.04,7.00,Blockmason Credit Protocol USD,0,0,0,2.440000e-03,283474.0,9.0,Blockmason1,the credit protocol whitepaper v blockmason te...


In [19]:
df_all = pd.merge(df, tfidf_df, left_index= True, right_index = True)
df_all.drop(columns= ['File Name'], inplace = True)
df_all.drop(df[df['Price (Intraday)'] > 500].index, inplace= True)
df_all.reset_index(inplace=True)
df_all.drop(columns = 'index',axis=1, inplace=True)

In [20]:
df_all

,Successful (Price),Successful (Market Cap),Successful (Volume),Name,File Size (MB),Number of Pages,Word Count,Flesch Reading Ease,New Dale-Chall,Crypto Name,...,ﬁfth,ﬁnalised,ﬁnalization,ﬁred,ﬁrmly,ﬁscal,ﬁxedpoint,ﬂagged,ﬂoat,ﬂuid
0,0,0,0,WeTrust1,1.342260,40,12504,43.02,7.33,WeTrust USD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,Libra4,0.293671,41,17474,54.42,6.51,Libera USD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,B2BX1,15.404950,42,1,121.22,19.48,X USD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,1,Bytom1,0.374002,17,4536,34.15,7.79,Bytom USD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,Cindicator1,1.123809,37,15509,42.92,7.08,Cindicator USD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,0,0,1,CPChain1,1.195266,34,7203,40.28,6.96,CPChain USD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
542,1,1,1,Presearch2,5.291211,39,12904,41.70,7.16,Presearch USD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
543,0,0,0,Electra1,0.624846,34,5508,37.13,9.05,Electra USD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
544,0,0,0,Blockmason1,3.355893,31,13640,49.04,7.00,Blockmason Credit Protocol USD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
meta_data = df_all.iloc[:, 4:13]
meta_data.drop(columns= 'Crypto Name', inplace=True)

In [66]:
meta_data

,File Size (MB),Number of Pages,Word Count,Flesch Reading Ease,New Dale-Chall,Abstract Int,Summary Int,Reference Int
0,1.342260,40,12504,43.02,7.33,1,0,1
1,0.293671,41,17474,54.42,6.51,1,0,1
2,15.404950,42,1,121.22,19.48,0,0,0
3,0.374002,17,4536,34.15,7.79,1,0,1
4,1.123809,37,15509,42.92,7.08,0,0,0
...,...,...,...,...,...,...,...,...
541,1.195266,34,7203,40.28,6.96,0,0,0
542,5.291211,39,12904,41.70,7.16,1,0,0
543,0.624846,34,5508,37.13,9.05,1,1,0
544,3.355893,31,13640,49.04,7.00,0,0,0


In [41]:
tfidf_data = df_all.iloc[:, 17:1582]
tfidf_data

,aarhus,abbreviate,abdelmalek,abound,abstracts,abu,accelerates,accessories,acclaim,accommodates,...,ﬁfth,ﬁnalised,ﬁnalization,ﬁred,ﬁrmly,ﬁscal,ﬁxedpoint,ﬂagged,ﬂoat,ﬂuid
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
mix_data = pd.merge(meta_data, tfidf_data, left_index= True, right_index= True)
mix_data

,File Size (MB),Number of Pages,Word Count,Flesch Reading Ease,New Dale-Chall,Abstract Int,Summary Int,Reference Int,aarhus,abbreviate,...,ﬁfth,ﬁnalised,ﬁnalization,ﬁred,ﬁrmly,ﬁscal,ﬁxedpoint,ﬂagged,ﬂoat,ﬂuid
0,1.342260,40,12504,43.02,7.33,1,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.293671,41,17474,54.42,6.51,1,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15.404950,42,1,121.22,19.48,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.374002,17,4536,34.15,7.79,1,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.123809,37,15509,42.92,7.08,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,1.195266,34,7203,40.28,6.96,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
542,5.291211,39,12904,41.70,7.16,1,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
543,0.624846,34,5508,37.13,9.05,1,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
544,3.355893,31,13640,49.04,7.00,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Meta Prediction

## With Flesch Reading Ease

In [131]:
X = meta_data
y = df_all['Successful (Price)']

## Without Flesch Reading Ease

In [180]:
meta_data_1 = meta_data
meta_data_1.drop(columns='Flesch Reading Ease', inplace=True)

In [181]:
meta_data_1

,File Size (MB),Number of Pages,Word Count,New Dale-Chall,Abstract Int,Summary Int,Reference Int
0,1.342260,40,12504,7.33,1,0,1
1,0.293671,41,17474,6.51,1,0,1
2,15.404950,42,1,19.48,0,0,0
3,0.374002,17,4536,7.79,1,0,1
4,1.123809,37,15509,7.08,0,0,0
...,...,...,...,...,...,...,...
541,1.195266,34,7203,6.96,0,0,0
542,5.291211,39,12904,7.16,1,0,0
543,0.624846,34,5508,9.05,1,1,0
544,3.355893,31,13640,7.00,0,0,0


In [182]:
X = meta_data_1
y = df_all['Successful (Price)']

In [183]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### KNN Classifier

In [184]:
knn_clf = KNeighborsClassifier(n_neighbors= 4)
knn_clf.fit(X_train, y_train)
knn_predict = knn_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, knn_predict))
print(metrics.classification_report(y_test, knn_predict))

[[61 22]
 [55 26]]
              precision    recall  f1-score   support

           0       0.53      0.73      0.61        83
           1       0.54      0.32      0.40        81

    accuracy                           0.53       164
   macro avg       0.53      0.53      0.51       164
weighted avg       0.53      0.53      0.51       164



### Decision Tree Classifier

In [185]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
dt_predict = dt_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, dt_predict))
print(metrics.classification_report(y_test, dt_predict))

[[40 43]
 [32 49]]
              precision    recall  f1-score   support

           0       0.56      0.48      0.52        83
           1       0.53      0.60      0.57        81

    accuracy                           0.54       164
   macro avg       0.54      0.54      0.54       164
weighted avg       0.54      0.54      0.54       164



### SGD Classifier

In [186]:
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train, y_train)
sgd_predict = sgd_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, sgd_predict))
print(metrics.classification_report(y_test, sgd_predict))

[[ 3 80]
 [ 0 81]]
              precision    recall  f1-score   support

           0       1.00      0.04      0.07        83
           1       0.50      1.00      0.67        81

    accuracy                           0.51       164
   macro avg       0.75      0.52      0.37       164
weighted avg       0.75      0.51      0.37       164



### MLP Classifier

In [187]:
mlp_clf = MLPClassifier()
mlp_clf.fit(X_train, y_train)
mlp_predict = mlp_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, mlp_predict))
print(metrics.classification_report(y_test, mlp_predict))

[[83  0]
 [81  0]]
              precision    recall  f1-score   support

           0       0.51      1.00      0.67        83
           1       0.00      0.00      0.00        81

    accuracy                           0.51       164
   macro avg       0.25      0.50      0.34       164
weighted avg       0.26      0.51      0.34       164



/Users/alisdghnia/.conda/envs/pythonProject/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alisdghnia/.conda/envs/pythonProject/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alisdghnia/.conda/envs/pythonProject/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

### Logistic Regression

In [188]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
logreg_predict = log_reg.predict(X_test)
print(metrics.confusion_matrix(y_test, logreg_predict))
print(metrics.classification_report(y_test, logreg_predict))

[[42 41]
 [36 45]]
              precision    recall  f1-score   support

           0       0.54      0.51      0.52        83
           1       0.52      0.56      0.54        81

    accuracy                           0.53       164
   macro avg       0.53      0.53      0.53       164
weighted avg       0.53      0.53      0.53       164



In [189]:
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)
nb_predict = nb_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, nb_predict))
print(metrics.classification_report(y_test, nb_predict))

[[52 31]
 [38 43]]
              precision    recall  f1-score   support

           0       0.58      0.63      0.60        83
           1       0.58      0.53      0.55        81

    accuracy                           0.58       164
   macro avg       0.58      0.58      0.58       164
weighted avg       0.58      0.58      0.58       164



# TFIDF Prediction

In [190]:
X = tfidf_data
y = df_all['Successful (Price)']

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### KNN Classifier

In [192]:
knn_clf = KNeighborsClassifier(n_neighbors= 4)
knn_clf.fit(X_train, y_train)
knn_predict = knn_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, knn_predict))
print(metrics.classification_report(y_test, knn_predict))

[[75  8]
 [78  3]]
              precision    recall  f1-score   support

           0       0.49      0.90      0.64        83
           1       0.27      0.04      0.07        81

    accuracy                           0.48       164
   macro avg       0.38      0.47      0.35       164
weighted avg       0.38      0.48      0.35       164



### Decision Tree Classifier

In [193]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
dt_predict = dt_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, dt_predict))
print(metrics.classification_report(y_test, dt_predict))

[[68 15]
 [63 18]]
              precision    recall  f1-score   support

           0       0.52      0.82      0.64        83
           1       0.55      0.22      0.32        81

    accuracy                           0.52       164
   macro avg       0.53      0.52      0.48       164
weighted avg       0.53      0.52      0.48       164



### SGD Classifier

In [194]:
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train, y_train)
sgd_predict = sgd_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, sgd_predict))
print(metrics.classification_report(y_test, sgd_predict))

[[33 50]
 [38 43]]
              precision    recall  f1-score   support

           0       0.46      0.40      0.43        83
           1       0.46      0.53      0.49        81

    accuracy                           0.46       164
   macro avg       0.46      0.46      0.46       164
weighted avg       0.46      0.46      0.46       164



### MLP Classifier

In [195]:
mlp_clf = MLPClassifier()
mlp_clf.fit(X_train, y_train)
mlp_predict = mlp_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, mlp_predict))
print(metrics.classification_report(y_test, mlp_predict))

[[53 30]
 [49 32]]
              precision    recall  f1-score   support

           0       0.52      0.64      0.57        83
           1       0.52      0.40      0.45        81

    accuracy                           0.52       164
   macro avg       0.52      0.52      0.51       164
weighted avg       0.52      0.52      0.51       164



### Logistic Regression

In [196]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
logreg_predict = log_reg.predict(X_test)
print(metrics.confusion_matrix(y_test, logreg_predict))
print(metrics.classification_report(y_test, logreg_predict))

[[56 27]
 [54 27]]
              precision    recall  f1-score   support

           0       0.51      0.67      0.58        83
           1       0.50      0.33      0.40        81

    accuracy                           0.51       164
   macro avg       0.50      0.50      0.49       164
weighted avg       0.50      0.51      0.49       164



### NB Classifier

In [197]:
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)
nb_predict = nb_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, nb_predict))
print(metrics.classification_report(y_test, nb_predict))

[[53 30]
 [50 31]]
              precision    recall  f1-score   support

           0       0.51      0.64      0.57        83
           1       0.51      0.38      0.44        81

    accuracy                           0.51       164
   macro avg       0.51      0.51      0.50       164
weighted avg       0.51      0.51      0.50       164



# All Data Prediction

In [198]:
X = mix_data
y = df_all['Successful (Price)']

In [199]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### KNN Classifier

In [200]:
knn_clf = KNeighborsClassifier(n_neighbors= 4)
knn_clf.fit(X_train, y_train)
knn_predict = knn_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, knn_predict))
print(metrics.classification_report(y_test, knn_predict))

[[62 21]
 [55 26]]
              precision    recall  f1-score   support

           0       0.53      0.75      0.62        83
           1       0.55      0.32      0.41        81

    accuracy                           0.54       164
   macro avg       0.54      0.53      0.51       164
weighted avg       0.54      0.54      0.51       164



### Decision Tree Classifier

In [201]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
dt_predict = dt_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, dt_predict))
print(metrics.classification_report(y_test, dt_predict))

[[54 29]
 [46 35]]
              precision    recall  f1-score   support

           0       0.54      0.65      0.59        83
           1       0.55      0.43      0.48        81

    accuracy                           0.54       164
   macro avg       0.54      0.54      0.54       164
weighted avg       0.54      0.54      0.54       164



### SGD Classifier

In [202]:
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train, y_train)
sgd_predict = sgd_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, sgd_predict))
print(metrics.classification_report(y_test, sgd_predict))

[[81  2]
 [81  0]]
              precision    recall  f1-score   support

           0       0.50      0.98      0.66        83
           1       0.00      0.00      0.00        81

    accuracy                           0.49       164
   macro avg       0.25      0.49      0.33       164
weighted avg       0.25      0.49      0.33       164



### MLP Classifier

In [203]:
mlp_clf = MLPClassifier()
mlp_clf.fit(X_train, y_train)
mlp_predict = mlp_clf.predict(X_test)
print(metrics.confusion_matrix(y_test, mlp_predict))
print(metrics.classification_report(y_test, mlp_predict))

[[11 72]
 [ 4 77]]
              precision    recall  f1-score   support

           0       0.73      0.13      0.22        83
           1       0.52      0.95      0.67        81

    accuracy                           0.54       164
   macro avg       0.63      0.54      0.45       164
weighted avg       0.63      0.54      0.44       164



### Logistic Regression

In [204]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
logreg_predict = log_reg.predict(X_test)
print(metrics.confusion_matrix(y_test, logreg_predict))
print(metrics.classification_report(y_test, logreg_predict))

[[39 44]
 [33 48]]
              precision    recall  f1-score   support

           0       0.54      0.47      0.50        83
           1       0.52      0.59      0.55        81

    accuracy                           0.53       164
   macro avg       0.53      0.53      0.53       164
weighted avg       0.53      0.53      0.53       164

